#  "Upgrading" Florida Tech's CAPP Reports

Adam Lastowka

## The Problem

Florida Tech's course catalog is pretty good. It's got nice hyperlinking, helpful popups, and a reliable search system. It even includes degree requirements for various majors. Let's see a screenshot:

<img src="freshman_year.png" width=800px>

Cool! Not too hard to follow, and very nice and interactive.

But what if I want to check which of those degree requirements I'm meeting?

Florida Tech's PAWS system for students has a handy tool just for that -- CAPP Reports (Curriculum, Advising, and Program Planning). Students can generate "reports" that tell them which requirements they need to meet / have met so far. Let's see how it looks:

<img src="course_report.png" width=800px>

Bleh. And this is just what I could fit in a screenshot -- these tables go on for pages. They aren't linked to the course catalog, so while planning courses, you need to have multiple tabs open while switching back and forth, double-checking that you entered the CRN right.

What I'd like to have is something like this:

<img src="https://www.colorado.edu/aps/sites/default/files/block/astrophysics_flowchart.jpg" width=800px>

As a very visual person, a diagram like this one from UCol Boulder is what I need in order to properly think about my courses. The list in the course catalog is nice, but I like flowcharts -- especially ones that automatically track my progress.

## The Solution

Yes, I could just draw a flowchart for the Physics major. This would be simple and easy. Unfortunately, I am a computer programmer, and consequently duty-bound to automate any menial task I encounter.

In this document I'll develop a procedure for transforming a CAPP report into a personalized, readable course dependency flowchart constructed from publicly available data.

First we'll need Florida Tech's course catalog.

## Extracting the Catalog

1. The course catalog does not have an API, and is stored in its entirety on a series of webpages found at https://catalog.fit.edu/content.php?catoid=12&navoid=551. Download the .html files of all 26 of these pages and save them.
2. Combine the pages together. Open a linux shell (I'm on Windows, so I used WLS Ubuntu) and run `ls -tQ *.html | xargs cat > concat.html`
3. This page contains the course names, but not their descriptions, which are only shown when a course name is clicked on. Thankfully, this page *does* contain links to individual course pages with these descriptions on them. This command extracts the links to these locations (first grep gets lines, second gets links, then sed removes those pesky "amp;"s):
```bash
    grep -e $'<td class=\"width\">.&nbsp;\t\t\t\t<a href=\"' concat.html | grep -Eo 'https://[^\"]+' | sed 's/\&amp;/\&/g' > links.txt
```
4. We now have a list of links to every course description in the catalog (careful, these description pages can change frequently). Move `links.txt` to new, empty folder.
5. Back in linux, run `wget -i links.txt` (sorry Florida Tech servers)
6. Process the HTML files...

## Extracting the Course Network (with Python)

### Libraries

I'm managing my environment with Anaconda running Python 3.8.
It might be annoying on windows. The Anaconda Navigator can help.
`distinctipy` and `pyvis` aren't part of Conda, so you'll have to install them manually. Don't get `pyvis` confused with `pyviz`, which is an entirely different library (this cost me far too much time).

In [28]:
from os import walk
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import re
from graphviz import Digraph
import holoviews as hv
from holoviews import opts
from pyvis.network import Network
from distinctipy import distinctipy
import math
import json

### The Course Class (Class Class?)

Contains all data regarding a course, sans any fluff text after the course description but not in the course "requirements" section.

In [29]:
class Course:
    def __init__(self):
        self.page_num = 0

        self.course_num = 0 # 1150
        self.course_code = '' # 'EGN'
        self.course_id = '' # 'EGN1150'
        self.course_name = '' # 'Combined Machine Shop Certification'

        self.credit_hours = '' # can be a range, like 1-3
        self.description = ''

        self.requirement = ''

        self.prerequisites = []
        self.corequisites = []
        self.recommended = []
        self.complements_courses = []
        
        # the following three values aren't used till much later in this notebook
        self.met = False # has this course been taken?
        self.in_major = False # is this course listed in the major reqs?
        self.req_attrs = []

        # Possible values: CC HU SS LA Hon Q CL
        self.tags = []

    def print_blurb(self):
        print(self.course_code + ' - ' +
              str(self.course_num) + ' ' +
              self.course_name +
              '\nCredit Hours: ' +
              str(self.credit_hours))
        print('Requirement: ' + self.requirement)
        print('Corequisites: ' + ' '.join(self.corequisites))
        print('Prerequisites: ' + ' '.join(self.prerequisites))
        print('Recommended: ' + ' '.join(self.recommended))
        print('This course complements: ' + ' '.join(self.complements_courses))
        print('\n')

### Regex Helper Functions

Parsing HTML with regex is going to be messy. It's usually a BIG no-no, but I'm allowed do it here because I know the exact format of all the HTML I'm working with. The following code is **very specific** to the data I have. It won't be pretty, but if I wanted pretty, I'd be using `BeautifulSoup` or some other HTML parser.

In [30]:
# searches target for a substring in between start and end
# this code actually isn't specific to this project at all... :P
def in_between(target, start, end):
    return re.search(start + '(.*)' + end, target).group(1)

In [31]:
# removes all .HTML tags (simple)
def remove_tags(target):
    return re.sub('<[^<]+?>', '', target)

In [32]:
def reformat_list(l):
    # add spaces so that the parenthesis are recognized seperate from coursenames
    # have fun reading this line lmao
    s = re.sub('\(', '( ', re.sub('\)', ' )', ' '.join(l))).strip()

    # change all 'BME 3081' to 'BME3081'
    o = re.sub('(?<=[A-Z]{3})\s(?=[0-9]{4})', '', s).split(' ')
    
    # remove empties
    return list(filter(None, o))

`walk()` is a very handy function.

In [33]:
def get_filenames(path):
    f = []
    for(dirpath, dirnames, filenames) in walk(path):
        f.extend(filenames)
        return f

### Parsing HTML

This is where things start to get messy. This code only works on the 2021 course catalog I downloaded from Florida Tech, and will probably need to be updated if the catalog changes or is reformatted. I'm handling a lot of edge cases here. The only way definitive solution to this would be a ML-based approach (god forbid) or just access to the registrar's database itself.

Though if I had that, this process would look very different.

In [34]:
def parse_html(path_to_folder):
    names = get_filenames(path_to_folder)
    try:
        names.remove('links.txt')
    except ValueError:
        pass
    course_list = []
    for html_path in names:

        c = Course()
        c.page_num = html_path[-5:]

        f = open(path_to_folder + html_path, 'r', encoding='utf8')
        lines = [line.rstrip() for line in f]

        # We've already opened an HTML file, now we need to scan through it until we find the line with the course
        # information on it.
        # IMPORTANT: This doesn't always work. For MAR 6899, the data was broken by two newline characters for some
        #            reason. I manually corrected this in the file. Proper HTML parsing would fix this.
        line = ''
        for s in lines:
            if '<p><h1 id=\'course_preview_title\'>' in s:
                line = s[51:] # crop out initial junk
                break

        # CN and dept. code are always the title of the box
        c.course_num = int(line[4:8]) # 4302
        c.course_code = line[:3]      # 'AVT'
        c.course_id = c.course_code + str(line[4:8])

        # split things up by linebreaks for manageability
        sections = line.split('<br>')

        c.course_name = sections[0].split('</h1>')[0][9:] # 'Aviation Safety Analysis'

        # I'm removing the <p> tags because a couple of courses, namely PSY 6550 and
        # MAR 6899, are formatted differently and have <p> tags in their HTML. What's up with those?
        c.credit_hours = remove_tags(in_between(re.sub('<p>', '', sections[0]), '</h1><strong>', '<hr>'))[14:]

        c.description = sections[0].split('<hr>')[1]
        # Some of the descriptions say "complements [some other course]" or "builds on [course]"
        # I use a catch-all here to grab those instances
        if 'complements' in c.description.lower():
            c.complements_courses = re.findall('[A-Z][A-Z][A-z] [0-9][0-9][0-9][0-9]', c.description)

        #print(sections[1])
        for i in range(1, len(sections)):
            # remove tags, strip, and remove non-breaking space character escapes
            sec_notag = re.sub('&#160;', '', remove_tags(sections[i]).lstrip())

            # It's one of those lines that looks like "(CC) (HU/SS) (LA) (Hon)"
            if re.sub('Hon', 'H', sec_notag).isupper():
                tagcat = re.sub('\(|\)', ' ', sections[i])
                # tagcat = re.sub('/', ' ', tagcat) # uncomment to remove the bipartite tags (like 'HU/SS')

                # remove empty strings
                c.tags = list(filter(None, tagcat.split(' ')))

            elif sec_notag.startswith('Requirement'):
                c.requirement = sec_notag.split(':')[1][1:]

            elif sec_notag.startswith('Prerequisite'):
                
                # get the junk out
                sec_notag = re.sub(',', '', sec_notag)
                sec_notag = re.sub('\xa0', ' ', sec_notag)
                
                # ECE3331 has a typo in this section (no second space in 'ECE 3331or')
                # fix those sorts of errors here
                
                # before we sub out the "OR"s, we need to sub out the "cORequisites" (this created some errors, heh)
                sec_notag = re.sub('Corequisite', ' CRQ ', sec_notag)
                sec_notag = re.sub('or', ' or ', sec_notag)
                sec_notag = re.sub('and', ' and ', sec_notag)
                
                sec_notag = re.sub('/ {2,}/g', ' ', sec_notag)

                # split the prerequisite requirements
                # e.g. '(PHY 0000 and MTH 0001) or BIO 1111'.split(' ')
                # remove all extraneous strings from array
                prereq_cmds = sec_notag.split(':')[1].split(' ')[1:-1]
                while not prereq_cmds[-1]:
                    del prereq_cmds[-1]
                c.prerequisites = prereq_cmds

                # corequisites are stored on the same line as prerequisites, parse them similarly
                if 'CRQ' in sec_notag:
                    # slightly different slicing for formatting reasons
                    coreq_cmds = sec_notag.split(':')[2].split(' ')[1:]
                    while not coreq_cmds[-1]:
                        del coreq_cmds[-1]
                    c.corequisites = coreq_cmds

            elif sec_notag.startswith('Recommended'):
                sec_notag = re.sub(',', '', sec_notag)
                sec_notag = re.sub('\xa0', ' ', sec_notag)
                # this section has less consistent formatting, so we have to walk through it line-by-line
                if ":" in sec_notag:
                    part = sec_notag.split(':')[1]
                    starr = re.findall('(?:to|or|and).[A-Z]{3}\s[0-9]{4}', part)
                    if len(starr) > 0:
                        if starr[0].startswith('to '):
                            starr[0] = starr[0][3:]
                        rec_cmds = []
                        for s in starr:
                            rec_cmds += s.split(' ')
                    c.recommended = rec_cmds
                else:
                    # yeah, this is a possibility. For just one course. :|
                    c.recommended = [sec_notag]

        c.complements_courses = reformat_list(c.complements_courses)
        c.corequisites = reformat_list(c.corequisites)
        c.prerequisites = reformat_list(c.prerequisites)
        c.recommended = reformat_list(c.recommended)

        course_list.append(c)
    return course_list

### Network Construction

I think that all things considered, the above code does a great job of getting course information from the HTML. The Course list is modular enough to stand on its own and be used for other means -- making the network is a secondary analysis step.

In [35]:
# For meaningful coloring
def unique_depts(course_list):
    return list(set([x.course_code for x in course_list]))

def col_to_hex(x):
    return '#{0:02x}{1:02x}{2:02x}'.format(max(0, min(int(x[0]*175.0 + 80.0), 255)), max(0, min(int(x[1]*175.0 + 80.0), 255)), max(0, min(int(x[2]*175.0 + 80.0), 255)))

In [36]:
# Dumb wrapper block made for show_network.
# Warning: This whole cell of code is inefficient, hacky, and messy.
# A lot of bugfixing and database correction happened here. I'm going to leave it
# as-is for now, since this visualization is tangential to the project.

def add_node_group(data, DG, dept_list, dept_color_map, course_list, courses_pos):
    xp = 0
    yp = 0
    # Add an edge going to every node mentioned, no matter where it's mentioned.
    # Ignore the operators, too: [(), and, or]
    for x in data:
        if len(x) == 7 and x not in DG.nodes() and x[:3] in dept_list:
            if courses_pos != None:
                xp = courses_pos[x]['x']
                yp = courses_pos[x]['y']
            DG.add_node(x, shape='box', color=col_to_hex(dept_color_map[x[:3]]), title="No Data!", x=xp, y=yp)

def show_network(course_list, courses_pos = None):
    DG = nx.DiGraph()
    
    dept_list = unique_depts(course_list)
    
    # It's been a while since I touched Python. I forgot how fun one-liners like these are.
    dept_color_map = dict(zip(dept_list, distinctipy.get_colors(len(dept_list))))
    course_map = dict(zip([x.course_id for x in course_list], course_list))
    
    for c in course_list:
        if len(c.prerequisites) + len(c.corequisites) + len(c.recommended) + len(c.complements_courses) > 0:
            xp = 0
            yp = 0
            if courses_pos != None:
                xp = courses_pos[c.course_id]['x']
                yp = courses_pos[c.course_id]['y']
            DG.add_node(c.course_id, shape='box', dept=c.course_code, color=col_to_hex(dept_color_map[c.course_code]), title="No Data!", x=xp, y=yp)
            
            add_node_group(c.prerequisites, DG, dept_list, dept_color_map, course_list, courses_pos)
            add_node_group(c.corequisites, DG, dept_list, dept_color_map, course_list, courses_pos)
            add_node_group(c.recommended, DG, dept_list, dept_color_map, course_list, courses_pos)
            add_node_group(c.complements_courses, DG, dept_list, dept_color_map, course_list, courses_pos)
            
            for x in c.prerequisites:
                if len(x) == 7:
                    DG.add_edge(c.course_id, x, title='Prerequisite')
            for x in c.corequisites:
                if len(x) == 7:
                    DG.add_edge(c.course_id, x, title='Corequisite')
            for x in c.recommended:
                if len(x) == 7:
                    DG.add_edge(c.course_id, x, title='Recommended Knowledge')
            for x in c.complements_courses:
                if len(x) == 7:
                    DG.add_edge(c.course_id, x, title='Complements')
    
    # Hey, guess what: AEE3150 references courses that DON'T EXIST. I think they messed up when typing the code?
    # this is stupid the data is stupid this whole entire function is stupid
    # duuuuuuuuuuuuuuhhhhhhhhhhhhh
    DG.nodes['MAE3083']['x'] = DG.nodes['AEE3150']['x']
    DG.nodes['MAE3083']['y'] = DG.nodes['AEE3150']['y']
    DG.nodes['MAE3161']['x'] = DG.nodes['AEE3150']['x']
    DG.nodes['MAE3161']['y'] = DG.nodes['AEE3150']['y']
    
    # You know, maybe there isn't a real database. Maybe it's just a bunch of unlinked plaintext in an excel file.
    # That would explain all the bad entries, at least.
    for x in DG.nodes():
        if x in course_map:
            DG.nodes[x]['title'] = course_map[x].course_name
        
    # display with pyvis for interactibility
    g=Network(height=900, width=1400, notebook=True, directed=True)
    g.force_atlas_2based()
    g.set_edge_smooth('continuous')
    
    # Hrm. I'd like physics to be on by default, but then visjs tries to stabilize the network,
    # even though I turn that off here. Fix in HTML?
    g.toggle_physics(False)
    g.toggle_stabilization(courses_pos==None)
    
    g.from_nx(DG) # I do love how friendly Python can be (when you're not installing modules, that is)
    g.show_buttons('physics')
    g.show("grph.html")
    # Wow, am using these comments as a blogging platform?
    # Actually, I think I'm just procrastinating writing the PDAG logic.
    # beebeebooboo
    # okay, fine
    return DG

In [37]:
print('parsing...')
CL = parse_html('webpages/descpages/') # This can take a while when run for the first time
print('making network...')

# I exported all the positions so we don't have to wait for the network to stabilize.
# I got them with ForceAtlas2 and some manual dragging of components out of local minima

saved_pos = json.loads(open('full_network_positions.json', 'r').read())
DG = show_network(CL, saved_pos)
print('done.')

parsing...
making network...
done.


### First Impressions

NOTE: If you don't want to bother installing the libraries required to run this notebook, you can view the network [here](grph.html).

Let's see what we're working with:

<img src="full_network_img_2.png">

Neat!

Nodes are courses, edges are relationships between them. Colors are based on departament codes. Obviously it's a [DAG](https://en.wikipedia.org/wiki/Directed_acyclic_graph) even though the directions aren't visible here. We've got a lot of isolated vertices, plenty of small components, and one massive one. The dense cluster on the left is an interesting anomaly. We'll explore that later. For now we need to find a way to handle the logical operators in these prerequisite lists -- stuff like `(MTH0000 and MTH0001) or MTH0002`.

### PDAGs (Propositional Directed Acyclic Graphs)

Right now, edge data looks like this:

`[ '(', 'MTH0000', 'and', 'MTH0001', ')', 'or', 'MTH0002']`

In the image from the previous section, all the logical operators and parenthesis are completely ignored. We're losing data.

Enter [PDAGs](https://en.wikipedia.org/wiki/Propositional_directed_acyclic_graph): a means of representing logical expressions in graphs. The concept is fairly simple: A node for every operand and an edge for every operator. If we generate these graphs for every course, we can compose them with the course network (while allowing them to retain some labels pertaining to their type and function).

This isn't too hard to do, especially since our PDAGs are already simplified past negation normal form -- their operators are limited to just conjunction and disjunction. Also, since both `and` and `or` are associative and commutative, we'll use just one `or` node for cases like `MTH1001 or MTH1010 or MTH1702`.

#### Prepping the Data

The dataset has some ambiguous entries that don't use parenthesis despite having multiple operators.
Let's introduce an order of operations:

In [38]:
# Issue: not every entry containing multiple operators uses parenthesis (:|).
# Here are all the tricky cases:

# ['BIO5210', 'and', 'BME5300', 'or', 'CHE5300']
# ['(', 'CSE1400', 'or', 'MTH1000', 'or', 'MTH1001', 'or', 'MTH1002', 'or', 'MTH1010', 'or', 'MTH1020',
#       'or', 'MTH1603', 'or', 'MTH1701', 'or', 'MTH1702', 'or', 'MTH2001', 'or', 'MTH2010', 'or', 'MTH2051', 
#       'or', 'MTH2201', 'or', 'MTH3200', ')', 'or', '(', 'MTH1011', 'and', 'MTH1012', ')', 'and', 'PSY1411']
# ['BIO4101', 'and', 'BIO4110', 'or', 'BIO4111']
# ['CSE1502', 'or', 'CSE1503', 'or', 'CSE2050', 'and', 'MTH2201']
# ['BME3030', 'and', 'BME5300', 'or', 'CHE5300']
# ['BIO2110', 'or', 'BIO2111', 'and', 'BIO2301']
# ['BIO2110', 'or', 'BIO2111', 'and', 'BIO4010', 'or', 'BIO4011']
# ['CHE1091', 'and', 'CHE3260', 'or', 'CHM2002']
# ['BME3260', 'or', 'CHE3260', 'and', 'AEE3083', 'or', 'BME3081', 'and', 'MEE2024', 'or', 'CHE4568']
# ['MTH1002', 'or', 'MTH1020', 'and', 'BME3260', 'or', 'CHE3260', 'or', 'CSE2410', 'or', 'ECE3551']
# ['CSE1001', 'and', 'MTH2201', 'or', 'MTH3200']
# ['CSE1001', 'and', 'MTH2201', 'or', 'MTH3200']

# In all of these, it seems like OR precedes AND (in order of ops.)
# We'll assume that's the case across the board.

# First, we'll concatenate everything that is already grouped, so we can treat it as one element.
def concat_paren_groups(list_in):
    ls = []
    # Just gonna roll with a simple FSM here.
    in_parens = False
    paren_cat = []
    for x in list_in:
        if in_parens:
            if x == ')':
                in_parens = False
                # This is how I handle nested parenthesis. Since this function is already aggregating
                # everything in the parens, it might as well also send it back to the correction function.
                # It's sort of superfluous since nested parens never happen in the data, though...
                corrected = '~'.join(correct_parens_recur(paren_cat))
                ls.append('(~' + corrected + '~)') # the tilde is a temporary substitute for a space
                paren_cat = []
            else:
                paren_cat.append(x)
        elif x == '(':
            in_parens = True
        else:
            ls.append(x)
    return ls

# Next, we need to identify where to add parenthesis.

# Yes, it does call itself, but it does it through concat_paren_groups()
def correct_parens_recur(list_in):
    # loop backwards
    ls = list_in.copy()
    # deal with any existing grouping
    if any('(' in x for x in ls):
        ls = concat_paren_groups(ls)
    state='nul'
    state_ever_changed = False
    last_change = len(ls)-1
    for i in range(len(ls)-1, 0, -1):
        
        # if we find a different operator at our current position
        if ls[i] != state and (ls[i]=='and' or ls[i]=='or'):
            # ... and it's not the first real state we find
            if not state == 'nul':
                # insert parenthesis around any OR set that is on the same hierarchical level as an AND
                if state == 'or':
                    ls.insert(last_change+2, ')')
                    ls.insert(i+1, '(')
                last_change = i
                state_ever_changed = True
                
            state = ls[i]
    
    # throw the parens in if we reached the end of the list and the conditions are right
    if state_ever_changed and state == 'or':
        ls.insert(last_change+2, ')')
        ls.insert(0, '(')
    
    return ls

# anything not a course, operator, or parenthesis returns false
def valid_course_data(x):
    return bool(re.search('\(|\)|or|and|[A-Z]{3}[0-9]{4}', x))

# changes "( MTH1000 or )" to "( MTH1000 )"
# takes a list of strings as input though, not a string
def remove_degenerate_operators(list_in):
    list_out = []
    was_op = False
    for x in list_in:
        list_out.append(x)
        if was_op and x == ')':
            del list_out[-2]
        was_op = ('or' in x.lower() or 'and' in x.lower())
    return list_out

# changes "( ( ( MTH1000 ) ) ) and MTH1001" to "MTH1000 and MTH1001"
# takes a list of strings as input though, not a string
def remove_degenerate_parens(list_in):
    list_out = []
    list_tmp = list_in.copy()
    
    # loop so we get all the nested parenthesis
    # there is a way to get them in one pass (keep track of depth), but I don't feel like writing it
    has_degen = True
    while has_degen:
        cells_since_Lparen = 0
        to_add = []
        list_out = []
        has_degen = False
        for x in list_tmp:
            to_add.append(x)
            if x == '(':
                cells_since_Lparen = 0
            elif x == ')':
                # If we see a ')' 2 spaces after a '(', then the parenthesis only enclose
                # one value, so we can delete them.
                if cells_since_Lparen == 2:
                    del to_add[-3] # crazy how this works in Python
                    del to_add[-1]
                    has_degen = True
                list_out.extend(to_add)
                to_add = []
            cells_since_Lparen += 1
        list_out.extend(to_add)
        list_tmp = list_out.copy()
    return list_out

# wrapper function for correct_parens_recur
# also removes any non-course data and fixes some formatting issues
def fix_formatting(list_in):
    # remove degenerates
    ls = remove_degenerate_operators(list_in)
    ls = remove_degenerate_parens(ls)
    
    ls = correct_parens_recur(ls)
    
    # remove the whitespace placeholders ('~'), and any zero-width space characters ('\u200b')
    ls = re.sub('\u200b', '', re.sub('~', ' ', ' '.join(ls))).split(' ')
    
    # remove any 'background knowledge' or similar non-parsable entries
    ls = list(filter(valid_course_data, ls))
    # remove any trailing operators as a result of the previous filter
    if len(ls) > 0 and (ls[0] == 'and' or ls[0] == 'or'): del ls[0]
    if len(ls) > 0 and (ls[-1] == 'and' or ls[-1] == 'or'): del ls[-1]
    
    # get rid of operators and do another paren pass
    ls = remove_degenerate_operators(ls)
    ls = remove_degenerate_parens(ls)
    
    return ls

#### Algorithm Time

The purpose of this function is to create a small NetworkX structure that can be overlayed on our complete network, or a subset of it. It needs to generate PDAGs from a logical expression in infix notation. This expression may contain nested parenthesis, but its operators are limited to `and` and `or`. This task is somewhat analogous to that of an operator precedence parser.

In [39]:
# We'll use a recursive function
def parse_conns_recur(ls, root_node_name, edge_label='???'):
    DG = nx.DiGraph()
    node_type = 'NULL'
    if len(ls) == 0:
        DG.add_node(root_node_name)
        return DG
    if len(ls) == 1:
        # this case is only triggered if the root node has no logic
        DG.add_node(ls[0])
        DG.add_edge(root_node_name, ls[0], title=edge_label)
    else:
        in_parens = 0
        paren_cat = []
        to_add = []
        to_add_parens = []
        for x in ls:
            if x == '(':
                in_parens += 1
            
            if in_parens > 0:
                paren_cat.append(x)
                
            else:
                if len(x) == 7:
                    to_add.append(x)
                else:
                    node_type = x
            
            if x == ')':
                in_parens -= 1
                if in_parens == 0:
                    to_add_parens.append(paren_cat[1:-1])
                    paren_cat = []
        if len(to_add) > 0 or len(to_add_parens) > 0:
            DG.add_node(root_node_name)
            this_node_name = ' '.join(ls)
            DG.add_node(this_node_name, label=node_type.upper())
            DG.add_edge(root_node_name, this_node_name, title=edge_label)
        if len(to_add) > 0:
            for x in to_add:
                DG.add_node(x)
                DG.add_edge(this_node_name, x, title=edge_label)
        if len(to_add_parens) > 0:
            for x in to_add_parens:
                SG = parse_conns_recur(x, this_node_name, edge_label)
                DG = nx.compose(DG, SG)
    return DG

# Another wrapper to correct the parenthesis in our input list
def get_PDAG(ls, root_node_name, edge_label):
    filt_list = fix_formatting(ls)
    DG = parse_conns_recur(filt_list, root_node_name, edge_label)
    return DG

#### Displaying the Network (again)

We'll display it like how we did before, but a little neater this time.

In [40]:
# SG for... uh... supergraph?
SG = nx.DiGraph()

# this can take a few seconds
for x in CL:
    DG2 = get_PDAG(x.prerequisites, x.course_id, 'prerequisite')
    SG = nx.compose(SG, DG2)
    # complements_courses has no logical operators, it's just a list of courses, so we won't pass it to parse_conns

In [41]:
dept_list = unique_depts(CL)
# It's been a while since I touched Python. I forgot how fun one-liners like these are.
dept_color_map = dict(zip(dept_list, distinctipy.get_colors(len(dept_list), pastel_factor=0.85)))

def disp_PDAG(pdag_graph, remove_isolates=True):
    gn=Network(height=900, width=1400, notebook=False, directed=True)

    for x in pdag_graph.nodes():
        pdag_graph.nodes[x]['shape'] = 'box'
        if 'label' in pdag_graph.nodes[x]:
            if pdag_graph.nodes[x]['label'] == 'OR':
                pdag_graph.nodes[x]['label'] = ' '
                pdag_graph.nodes[x]['color'] = '#ffaaaa'
                pdag_graph.nodes[x]['shape'] = 'triangleDown'
            if pdag_graph.nodes[x]['label'] == 'AND':
                pdag_graph.nodes[x]['label'] = ' '
                pdag_graph.nodes[x]['color'] = '#aaffaa'
                pdag_graph.nodes[x]['shape'] = 'triangle'
        else:
            code = x[0:3]
            if code in dept_color_map:
                pdag_graph.nodes[x]['color'] = col_to_hex(dept_color_map[code])

    pdag_graph.remove_nodes_from(list(nx.isolates(pdag_graph)))
    gn.force_atlas_2based()
    gn.set_edge_smooth('continuous')
    gn.toggle_physics(False)
    gn.from_nx(pdag_graph)
    gn.show_buttons('physics')
    gn.show("grph_pdag.html")
    
disp_PDAG(SG)

### Second Impressions

NOTE: If you don't want to bother installing the libraries required to run this notebook, you can view the network [here](grph_pdag.html). Be sure to enable physics.

The network (this time of just prerequisites) has roughly the same large-scale structure:

<img src="prereq_pdag_broad.png" width=500px>

But if we zoom in, we can see there's a lot more going on at small scales.

<img src="prereq_pdag.png" width=800px>

The red, downward-facing triangles represent an `OR` operator, and the green upward-facing triangles represent an `AND` operator. With this, the network is now able to fully represent the logic in the course catalog data.

## Reformatting the CAPP Report

Now that we've constructed our course graph, we have the data necessary to make sense of the CAPP report.

### Extracting the Report

PAWS can display CAPP reports in three formats: Detailed, General, or Additional Information. I'll select the "detailed" version:

<img src="CAPP.png"></img>

And I'll manually download the .html page by hitting `ctrl+s` in my browser.

<img src="course_report.png" width=800px></img>

The data on these pages is stored in tables, so we should probably keep it in that format. Let's try using `pandas.read_html()` to get the information:

In [66]:
# read_html gives us a list of dataframes. The indices are as follows:
# [0]  - nothing useful
# [1]  - overall credit requirements
# [2]  - program information
# [3:-3] - actual course reqs.
# [-3] - in-progress courses (MAY NOT APPEAR)
# [-2] - courses not used    (MAY NOT APPEAR)
# [-1] - letter key (H - History, etc.)
CAPP_data = pd.read_html('webpages/CAPPpages/detailed 2.html')
print(CAPP_data[3].iloc[2, :]) # try printing a row

0                          Yes
1                          NaN
2                          NaN
3                          COM
4                          NaN
5                         1101
6                          NaN
7                       202008
8                          COM
9                         1101
10    Composition and Rhetoric
11                         NaN
12                       3.000
13                           T
14                           T
15                         NaN
Name: 2, dtype: object


Wow, that worked!

This doesn't usually happen!

Let's start crunching it.

In [43]:
credit_info = CAPP_data[1]
program_info = CAPP_data[2]
course_info = CAPP_data[3:]
# Saves all data to .csv for debugging
# Excel or google sheets is great for looking at all the info
pd.concat([credit_info, program_info, *course_info]).to_csv('webtest.csv', encoding='utf-8',sep='\t')

In [44]:
# checks if any strings in a dataframe contain a value
# mild TODO: vectorize this somehow
def is_in_df(df, my_str):
    for ri, row in df.iterrows():
        for ci, val in row.items():
            if my_str in str(val):
                return True
    return False

req_data = []
other_data = []
for table in course_info:
    # strip out all whitespace
    table = table.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    # look a couple code blocks below for a list of all the metadata that dataframes are tagged with
    
    if is_in_df(table, 'R - Currently Registered'):
        # this is the 'Source Code Key' table
        table.attrs['type'] = 'SCC'
        other_data.append(table)
    elif table.shape[1] == 7:
        # this is the 'In-Progress Courses' table
        table.attrs['type'] = 'IPC'
        other_data.append(table)
    elif table.shape[1] == 6:
        # this is the 'Courses Not Used' table
        table.attrs['type'] = 'CNU'
        other_data.append(table)
    elif table.shape[1] == 16:
        # this is one of the actual course tables
        # print('REQUIREMENTS FOUND: ' + str(table.shape[0]-3) + ' condition(s)')
        table.attrs['type'] = 'REQ'
        
        table_title = table.iloc[0, 3]
        
        # save table metadata in dataframe attributes
        req_infos = '-'.join(table_title.split('-')[:-1])
        table.attrs['req id'] = req_infos.split('(')[0].strip()
        table.attrs['credits'] = float(req_infos.split('(')[-1].split('credits')[0])
        table.attrs['met'] = not ('NOT' in table_title.split('-')[-1].upper())
        table.attrs['is semester'] = 'Sem1' in table_title or 'Sem2' in table_title
        if table.attrs['is semester']:
            i = -1
            sm = table.attrs['req id'].lower()
            if 'freshmen' in sm:
                i = 0
            elif 'sophomore' in sm:
                i = 2
            elif 'junior' in sm:
                i = 4
            elif 'senior' in sm:
                i = 6
            else:
                table.attrs['is semester'] = False
            
            if '2' in sm:
                i = i+1
            table.attrs['semester id'] = i
        
        req_data.append(table.iloc[2:-1 , :]) # These rows have nothing useful

We've successfully extracted the metadata from our tables, and removed all junk data. Now we need to actually parse our data. Thankfully, we already have a PDAG parser that takes input in the `['(', 'MTH2001', 'OR', 'MTH2011', ')']` form! We just need to adapt the present information to fit this standard.

Also, we have the handy `fix_formatting()` function from earlier, so we won't have to worry about parenthesis.

In [45]:
for i in range(0, len(req_data)):
    df = req_data[i]
    # I'm going to iterate through the dataframes.
    # This is an anti-pattern in Pandas, and generally frowned upon, but I think
    # vectorization would introduce unnecessary complexity into the code.
    
    # Once again, the formatting in the data is inconsistent.
        # There are a few weird cases on my report:
        #
        # 1: Every column says PHY3152/53 or something like that
        #    Sometimes it's "A and B", other times it's "A & B", or "A/B".
        #    By 'every', I mean in rule/subj/attrib/low/high.
        #    This appears to happen for all courses with labs.
        #    I'm pretty confident that this wasn't an intentional feature.
        #
        # 2: low/high are actually used (e.g. low:2000, high:4999)
        #    Each instance of this case is paired with one of the following cases:
        #
        # 3: Low/high are set to 2XXX and 4XXX, respectively
        #    I guess either case 2 or 3 wasn't working, so they put in the other but didn't remove the old one?
        #    Anyway, they mean the same thing. Probably safe to include both instances in the network, then
        #    remove the duplicate edge? I want to mess with the formatting as little as possible
        #
        # 4: The row does not specify a course range / CRN.
        #    This happens with elective requirements and other gen-ed stuff.
        #
        # 5: The row only contains a right parenthesis.
        #
        # THERE IS A HIGH CHANCE THAT ADDITIONAL CAPP REPORTS WILL HAVE CASES I HAVEN'T DOCUMENTED HERE
    
    statement = []
    for i in range(0, df.shape[0]):
        
        met =  df.iloc[i, 0]
        cond = df.iloc[i, 1]
        
        rule = df.iloc[i, 2]
        subj = str(df.iloc[i, 3])
        low  = str(df.iloc[i, 5])
        high = str(df.iloc[i, 6])
        
        course_id_2 = str(df.iloc[i, 8]) + str(df.iloc[i, 9])
        
        # if the locical operator column is non-empty at this row, add it to the string
        if str(cond).lower()!='nan':
            if ')' in cond:
                statement.append(')')
            if 'and' in cond.lower():
                statement.append('and')
            elif 'or' in cond.lower():
                statement.append('or')
            if '(' in cond:
                statement.append('(')
        
        # ------------------------ CASE HANDLING ------------------------ #
        # (see lengthy comment above this loop for details)
        
        # CASE 5 (rparen)
        if str(met).lower()=='nan':
            pass
            
            # CASE 1 (and)
        elif ('and' in low.lower()) or ('/' in low.lower()) or ('&' in low.lower()):
            # IMPORTANT: This assumes that this case only ever has one departament (no 'MTH1000 and PSY1000')
            # However, it can handle an arbitrary number of 'and's
            CRNs = list(set(re.findall('[0-9][0-9][0-9][0-9]', low + ' - ' + high + ' - ' + subj)))
            dept = list(set(re.findall('[A-Z][A-Z][A-Z]', low + ' - ' + high + ' - ' + subj)))[0]
            
            to_add = ['('] # put everything in a paren block since OOO prioritizes OR over AND
            for i in range(0, len(CRNs)):
                to_add.append(dept + CRNs[i])
                
                # here we fill the courselist's entries with new data we gathered from the CAPP report
                ck = next((c for c in CL if c.course_id == dept + CRNs[i]), None)
                if ck is not None:
                    # check if the course was taken / condition met
                    va = CL.index(ck)
                    CL[va].met |= ('yes' in met.lower())
                    CL[va].in_major = True
                    # copy the requirement metadata over to every node
                    CL[va].req_attrs.append(df.attrs)
                
                if i != len(CRNs) - 1:
                    to_add.append('or')
            to_add.append(')')
            statement.extend(to_add)
            
            # CASE 2 (XXXX)
        elif ('x' in low.lower()) or ('x' in high.lower()):
            # find course range by finding maximum and minimum possible course vals by substituting 'x' for 9 and 0
            mina = min(int(low.lower().replace('x', '0')), int(high.lower().replace('x', '0')))
            maxa = max(int(low.lower().replace('x', '9')), int(high.lower().replace('x', '9')))
            
            # iterate through courselist and find any courses in the range [mina, maxa], then add them
            good_courses = []
            for x in CL:
                if x.course_code == subj and x.course_num >= mina and x.course_num <= maxa:
                    good_courses.append(x.course_id)
                    
                    # entry filling like in case 1
                    va = CL.index(x)
                    CL[va].met |= ('yes' in met.lower() and x.course_id == course_id_2)
                    CL[va].in_major = True
                    CL[va].req_attrs.append(df.attrs)
            
            for i in range(0, len(good_courses)):
                statement.append(good_courses[i])
                if i != len(good_courses) - 1:
                    statement.append('or')
            
            # CASE 3 (low/high)
        elif low.isnumeric() and high.isnumeric():
            good_courses = []
            # iterate through courselist and find any courses in the range [low, high], then add them
            for x in CL:
                if x.course_code == subj and x.course_num >= int(low) and x.course_num <= int(high):
                    good_courses.append(x.course_id)
                    
                    # entry filling like in case 1
                    va = CL.index(x)
                    CL[va].met |= ('yes' in met.lower() and x.course_id == course_id_2)
                    CL[va].in_major = True
                    CL[va].req_attrs.append(df.attrs)
            
            # put it all in an 'or' block
            for i in range(0, len(good_courses)):
                statement.append(good_courses[i])
                if i != len(good_courses) - 1:
                    statement.append('or')
        
            # CASE 4 (no course range / num specified)
        elif str(rule).lower()!='nan' and not low.isnumeric() and not high.isnumeric():
            pass
        
            # DEFAULT CASE (CRN in low)
        else:
            # just copy the id over
            statement.append(subj + low)
            
            # entry filling like in case 1
            ck = next((c for c in CL if c.course_id == subj + low), None)
            if ck is not None:
                va = CL.index(ck)
                CL[va].met |= ('yes' in met.lower())
                CL[va].in_major = True
                CL[va].req_attrs.append(df.attrs)
            
    statement = fix_formatting(statement)
    df.attrs['statement'] = statement

In [46]:
# ALL DATAFRAME ATTRIBUTES:

# ATTRIBUTE KEY - EXAMPLE - DESCRIPTION

# 'type'        - 'SCC'/'IPC'/'CNU'/'REQ'       - source code key / in progress courses / courses not used / requirements
# 'req id'      - 'PhysicsBach-MTH-or-CSE'      - identifier for the requirement
# 'credits'     - 6.0                           - credits needed to fulfill this requirement
# 'met'         - True                          - has this requirement been met?
# 'is semester' - True                          - does this requirement correspond with a semester?
# 'semester id' - 5                             - value from 0-7 indicating which semester this requirement is for
# 'statement'   - ['MTH1001', 'and', 'MTH1000'] - the logic used to determine if the requirement has been fulfilled
        
#ept_list = unique_depts([x for x in CL if x.in_major])
#dept_color_map = dict(zip(dept_list, distinctipy.get_colors(len(dept_list), pastel_factor=0.9, colorblind_type='Tritanopia')))

SG_reqs = nx.DiGraph()
for x in req_data:
    q = get_PDAG(x.attrs['statement'], x.attrs['req id'], 'requirement')
    SG_reqs = nx.compose(SG_reqs, q)

for x in CL:
    if x.in_major:
        DG2 = get_PDAG(x.prerequisites, x.course_id, 'prerequisite')
        SG_reqs = nx.compose(SG_reqs, DG2)
        
for x in CL:
    if x.in_major:
        DG2 = get_PDAG(x.corequisites, x.course_id, 'corequisite')
        SG_reqs = nx.compose(SG_reqs, DG2)

for x in CL:
    if x.in_major:
        DG2 = get_PDAG(x.recommended, x.course_id, 'recommended')
        SG_reqs = nx.compose(SG_reqs, DG2)

for x in CL:
    if x.in_major:
        DG2 = get_PDAG(x.complements_courses, x.course_id, 'complements')
        SG_reqs = nx.compose(SG_reqs, DG2)
        
disp_PDAG(SG_reqs)

### Graduation Requirement Network

That was easier than I expected. My physics major CAPP report has been turned into a network:

<img src="physics_network.png" width=500px></img>

However, this network isn't very helpful. It is currently:
- **Hard to read.** The nodes are too small/spread out, and they lack useful information about the courses.
- **Hard to comprehend.** It might be difficult to understand how the OR/AND triangle nodes work.
- **Unorganized.** The courses should be arranged by semester in chronological order.
- **Overwhelming.** So much data is being displayed that it is difficult to grasp any meaningful insights from it.

Pyvis, our current visualization tool, is great for viewing the large-scale structure of a network, but it doesn't support any hierarchical layouts. We'll need a better tool to display these networks.

## Visualization

While working on this project, I've experimented with **Pyvis**, **Networkx**, **Graphviz**, and **Holoviz**/**Holoviews**. Out of the four, **Pyvis** and **Graphviz** created the most effective visualizations with the least hassle.

Still, they're not *easy* to use. Pyvis is essentially a bridge from Python to the larger **vis.js**, a comprehensive browser-based visualization library. The documentation for vis.js is great, but for Pyvis... not so much. Similarly, Graphviz is a CLI tool that I can use here because of a Python interfaces that connect me to the tool. The docs for this interface are also unsatisfactory.

Because of the shoddy documentation and my lower comfort level in Python, I'm going to export the data and write the visualizer in another language.

### Exporting

I could export to JSON through NetworkX, but I feel like cleaning things up a little before exporting.

In [47]:
node_export = {}
for n in SG_reqs.nodes(data=True):  
    node_properties = {
        'is logic'     : len(n[0].strip()) > 7,
        'is semester'  : False,
        'is operator'  : False,
        'operator'     : 'No operator!',
        'in major'     : False,
        'credits'      : 0,
        'name'         : 'No course name provided!',
        'code'         : 'No code provided!',
        'num'          : -1,
        'description'  : 'No description provided!',
        'met'          : False,
        'tags'         : [],
        'req ids'      : [], # ids of parent requirement groups
        'semesters'    : [], # indices of parent requirement semesters
        'color'        : '#FFFFFF'
    }
    
    if not node_properties['is logic']:
        crs = next((c for c in CL if c.course_id == n[0]), None)
        if crs is not None:
            node_properties['in major'] = crs.in_major
            node_properties['credits'] = crs.credit_hours
            node_properties['name'] = crs.course_name
            node_properties['code'] = crs.course_code
            node_properties['num'] = crs.course_num
            node_properties['description'] = crs.description
            node_properties['met'] = crs.met
            node_properties['tags'] = crs.tags
            
            # nodes could belong to multiple semesters
            for x in crs.req_attrs:
                if 'semester id' in x:
                    node_properties['semesters'].append(x['semester id'])
                node_properties['req ids'].append(x['req id'])
        node_properties['semesters'] = list(set(node_properties['semesters']))
    else:
        # So, yes, I'm pulling these values from the triangle visualization code from earlier.
        # Probably not the best idea. TODO: Store operator type data elsewhere.
        if n[1]['shape'] == 'box' and ('freshmen' in n[0].lower() or 'junior' in n[0].lower() or
                                       'sophomore' in n[0].lower() or 'senior' in n[0].lower()):
            node_properties['is semester'] = True
        if n[1]['shape'] == 'triangle':
            node_properties['operator'] = 'AND'
            node_properties['is operator'] = True
        if n[1]['shape'] == 'triangleDown':
            node_properties['operator'] = 'OR'
            node_properties['is operator'] = True
    
    if node_properties['code'] in dept_color_map:
        node_properties['color'] = col_to_hex(dept_color_map[node_properties['code']])
    
    node_export[n[0]] = node_properties

edge_export = []
for e in SG_reqs.edges(data=True):
    node_a = e[0]
    node_b = e[1]
    node_type = e[2]['title']
    # very inefficient! very readable... :3
    edg = {'start' : node_a, 'end' : node_b, 'type' : node_type}
    edge_export.append(edg)

# req data is already well-formatted
reqs_export = []
for x in req_data:
    reqs_export.append(x.attrs)

# JSON is very easy in Python
to_export = {'nodes' : node_export, 'edges' : edge_export, 'requirements' : reqs_export}
with open('visualizer/mynetwork.json', 'w') as outfile:
    json.dump(to_export, outfile)

print('Exported!')

Exported!


## Done!

The tough work, anyway.

The rest of this adventure happens in Javascript. JS is more prototype-OO-ish stuff, so documenting my process like I've done here doesn't really work. Also, the code is a mess since I wrote it in a rush.

Regardless of its quality, the JS program is a browser-based specialized hierarchical-but-also-force-based graph drawing tool. Most of the code is drawing-focused, but a significant portion of it is dedicated to translating the JSON from an edge list into a linked list. It has some quirks, but I love the way it looks, and it's actually helped me think about what courses I need to take!

You can view the end result [here](visualizer/index.html). Thanks for reading!